In [1]:
"""

To recreate the same results with the pseudo-random numbers simply run all the cells once in order from top to bottom

"""


import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.stats import norm
seed = 7654
rng = np.random.default_rng(seed)

# defining all the constants

pi = np.pi

qed_coupling = 1/129

qcd_coupling = 0.118

zboson_mass = 91.2

zboson_decaywidth = 2.5

weinberg_angle = 0.223

electron_charge = -1

upquark_charge = 2/3

downquark_charge = -1/3

upquark_isospin = 1/2

downquark_isospin = -1/2

number_of_quark_flavours = 5

number_of_qcd_colours = 3

conversion_factor = 3.89379656*10**8

kappa = 1/(4*weinberg_angle*(1-weinberg_angle))

electron_vectorcoupling = downquark_isospin - 2* electron_charge*weinberg_angle

upquark_vectorcoupling = upquark_isospin - 2* upquark_charge*weinberg_angle

downquark_vectorcoupling= downquark_isospin - 2* downquark_charge*weinberg_angle

In [2]:
uniform_distribution = 1/((zboson_mass + 3*zboson_decaywidth)**2-(zboson_mass - 3*zboson_decaywidth)**2)

def breit_wigner_dist(s):
    
    return 1/((s- zboson_mass**2)**2 + zboson_decaywidth**2 * zboson_mass**2)

#def inverse_bw_dist(x):
    
 #   return np.sqrt(1/x - zboson_mass**2 * zboson_decaywidth**2) + zboson_mass**2

def G(s):
    
    return - np.arctan((zboson_mass**2 - s)/(zboson_mass*zboson_decaywidth))/(zboson_mass*zboson_decaywidth)

def inverse_G(x):
    
    return zboson_mass**2 - zboson_decaywidth*zboson_mass* np.tan(- zboson_mass*zboson_decaywidth*x)
    

def chi_1(s): 

    return kappa* (s*(s- zboson_mass**2))/((s- zboson_mass**2)**2 + zboson_decaywidth**2 * zboson_mass**2)

def chi_2(s):

    return kappa**2 * (s**2)/((s- zboson_mass**2)**2 + zboson_decaywidth**2 * zboson_mass**2)
    
#defining the matrix element e^+ e^- --> qq

def matrix_element(s,cos_theta,quark_flavour):

    if (quark_flavour == 2 or quark_flavour ==5): 
        return (4*pi*qed_coupling)**2 * number_of_qcd_colours* ((1+ cos_theta**2)*(electron_charge**2*upquark_charge**2 + 2* electron_charge * upquark_charge* electron_vectorcoupling * upquark_vectorcoupling * chi_1(s) + (downquark_isospin**2+ electron_vectorcoupling**2)*(upquark_isospin**2 + upquark_vectorcoupling**2)*chi_2(s))+ cos_theta* (4* electron_charge*upquark_charge*downquark_isospin*upquark_isospin*chi_1(s)+ 8* downquark_isospin*electron_vectorcoupling* upquark_isospin* upquark_vectorcoupling*chi_2(s)))
    
    else:  
        return (4*pi*qed_coupling)**2 * number_of_qcd_colours* ((1+ cos_theta**2)*(electron_charge**2*downquark_charge**2 + 2* electron_charge * downquark_charge* electron_vectorcoupling * downquark_vectorcoupling * chi_1(s) + (downquark_isospin**2+ electron_vectorcoupling**2)*(downquark_isospin**2 + downquark_vectorcoupling**2)*chi_2(s))+ cos_theta* (4* electron_charge*downquark_charge*downquark_isospin*downquark_isospin*chi_1(s)+ 8* downquark_isospin*electron_vectorcoupling* downquark_isospin* downquark_vectorcoupling*chi_2(s)))

def differential_cross_section(s,cos_theta):

    summed_matrix_elements = 0
    
    for i in range(number_of_quark_flavours):
    
        quark_flavour = i+1
        
        summed_matrix_elements = summed_matrix_elements + matrix_element(s,cos_theta,quark_flavour)
    
    return conversion_factor *uniform_distribution* 1/(8*pi) *1/(4*pi) * 1/(2*s) *summed_matrix_elements

In [3]:
#calculating the normalization constant for breit wigner distribution
def mc_inte(function, low_limits, up_limits, number_of_samples):
    
    dimension = 1
    
    integration_volume = up_limits- low_limits
    
    samples = np.random.uniform(low_limits, up_limits, number_of_samples)
    
    function_values = function(samples)
    
    integral = integration_volume * np.mean(function_values)
    
    error = integration_volume * np.std(function_values) / np.sqrt(number_of_samples)
    
    return integral,error

N = 1000000
s_lowlimit = (zboson_mass - 3*zboson_decaywidth)**2
s_uplimit  = (zboson_mass + 3*zboson_decaywidth)**2

normalization_constant = mc_inte(breit_wigner_dist,s_lowlimit,s_uplimit,N)

In [4]:
def norm_breit_wigner_dist(s):
    
    return 1/normalization_constant[0]/((s- zboson_mass**2)**2 + zboson_decaywidth**2 * zboson_mass**2)

#function to get samples with the Breit-Wigner mapping

def sampling(invers_func,up_limits , low_limits,number_of_samples):
    
    A_tot = (up_limits-low_limits)*rng.uniform(0,1,number_of_samples)
    
    s_samples = invers_func(low_limits + A_tot)
    
    return s_samples


In [5]:
#monte carlo integrator with importance sampling
def mc_integrator(function,dist,integ_of_dist,inverse_dist, low_limits, up_limits, number_of_samples):
    
    dimension = len(low_limits)
    
    integration_volume = up_limits[0]- low_limits[0]
    
    G_lowlimit = integ_of_dist(low_limits[0])
    G_uplimit = integ_of_dist(up_limits[0])
    
    s_samples = sampling(inverse_dist,G_uplimit,G_lowlimit,number_of_samples) 
    
    samples = np.random.uniform(low_limits[1], up_limits[1], number_of_samples)
    
    function_values = function(s_samples,samples)
    
    dist_values = dist(s_samples)
    
    integral =  np.mean(function_values/dist_values) *4*pi 
    
    error =  np.std(function_values/dist_values) / np.sqrt(number_of_samples)* 4*pi 
    
    return integral,error

In [6]:
N = 10000
s_lowlimit = (zboson_mass - 3*zboson_decaywidth)**2
s_uplimit  = (zboson_mass + 3*zboson_decaywidth)**2

low_limits = np.array([s_lowlimit,-1,0])
up_limits = np.array([s_uplimit,1,2*pi])
# Calculate the integral using importance sampling
result = mc_integrator(differential_cross_section,norm_breit_wigner_dist,G,inverse_G, low_limits, up_limits, N)
print("Approximated integral using importance sampling:", result)



Approximated integral using importance sampling: (9910.211307995301, 29.098138528477985)


In [7]:
number_of_samples = [10,25,50,100,200,500,1000,2000,5000,7500,10000,20000,50000,100000]
integration_values_mcerror  = [mc_integrator(differential_cross_section,norm_breit_wigner_dist,G,inverse_G,low_limits,up_limits,i) for i in number_of_samples]
integration_values_mcerror = np.array(integration_values_mcerror)

with open('../Data_and_Plots/data1.1fPart1.txt', 'wb') as f:
    header = 'Number of Samples  MCerror\n'
    header_ascii = header.encode('ascii')
    f.write(header_ascii)
    for i in range(len(number_of_samples)):
        line = str(number_of_samples[i])+ ' ' + str(integration_values_mcerror[:,1][i]) + '\n'
        line_ascii = line.encode('ascii')
        f.write(line_ascii)